<a href="https://www.kaggle.com/code/girishkalyan/bp-prediction?scriptVersionId=222609827" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import warnings

In [43]:
warnings.filterwarnings("ignore")


In [44]:
train = pd.read_csv("/kaggle/input/playground-series-s5e2/train.csv")
train_extra = pd.read_csv("/kaggle/input/playground-series-s5e2/training_extra.csv")
train_data = pd.concat([train, train_extra], axis=0, ignore_index=True)

test_data = pd.read_csv("/kaggle/input/playground-series-s5e2/test.csv")



In [45]:
# Encode categorical features
categorical_features = ['Brand', 'Material', 'Size', 'Style', 'Color']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_encoded = encoder.fit_transform(train_data[categorical_features])
categorical_data = pd.DataFrame(categorical_encoded, index=train_data.index, columns=encoder.get_feature_names_out(categorical_features))


****Feature Extraction****

In [46]:
# Encode categorical features for test data
categorical_encoded_test = encoder.fit_transform(test_data[categorical_features])
categorical_data_test = pd.DataFrame(categorical_encoded_test, columns=encoder.get_feature_names_out(categorical_features))

In [47]:
numerical_features = ['id','Compartments', 'Weight Capacity (kg)']
binary_features = ['Laptop Compartment', 'Waterproof']


In [48]:
test_data[binary_features] = test_data[binary_features].replace({'Yes': 1, 'No': 0})


In [ ]:
train_data[binary_features] = train_data[binary_features].replace({'Yes': 1, 'No': 0})
test_data[binary_features] = test_data[binary_features].replace({'Yes': 1, 'No': 0})


In [ ]:
X_test = pd.concat([test_data[numerical_features + binary_features], categorical_data_test], axis=1)

In [ ]:
X_train = pd.concat([train_data[numerical_features + binary_features], categorical_data], axis=1)
Y_train = train_data[["Price"]]
X_test = pd.concat([test_data[numerical_features + binary_features], categorical_data_test], axis=1)

In [ ]:
display(X_train)

In [ ]:
X_train.drop('id', axis =1)

****Feature Scaling****

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Use IterativeImputer (more intelligent but still efficient)
imputer = IterativeImputer(max_iter=10, random_state=42)

X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)

print("Imputation complete using IterativeImputer!")


In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Use IterativeImputer (more intelligent but still efficient)
imputer = IterativeImputer(max_iter=10, random_state=42)

X_test = pd.DataFrame(imputer.fit_transform(X_test), columns=X_test.columns)

print("Imputation complete using IterativeImputer!")


In [ ]:
missing_count_per_column = X_test.isnull().sum()
print(missing_count_per_column)


In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
# Ensure your data is in NumPy array format
X_train = X_train.to_numpy() if hasattr(X_train, 'to_numpy') else X_train
Y_train = Y_train.to_numpy() if hasattr(Y_train, 'to_numpy') else Y_train
X_test = X_test.to_numpy() if hasattr(X_test, 'to_numpy') else X_test


**Random Forrest Initialization**

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
import tensorflow as tf

# Initialize the MirroredStrategy
strategy = tf.distribute.MirroredStrategy()

print(f"Number of devices: {strategy.num_replicas_in_sync}")


In [ ]:
with strategy.scope():
    model = RandomForestRegressor(
            n_estimators=100,            # Number of trees in the forest
            max_depth=None,              # Maximum depth of the tree (None allows full depth)
            min_samples_split=2,         # Minimum samples required to split an internal node
            min_samples_leaf=1,          # Minimum samples required to be a leaf node
            max_features='sqrt',         # Number of features to consider when looking for the best split
            bootstrap=True,              # Whether bootstrap samples are used when building trees
            random_state=42             # Random state for reproducibility
        
        )

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold


# Perform K-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
scores = []

for train_index, val_index in kf.split(X_train):
    # Split the data
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[val_index]

    # Train the model
    model.fit(X_train_fold, Y_train_fold)

    # Evaluate the model
    score = model.score(X_val_fold, Y_val_fold)
    print(f"Fold {fold_no} - R2 Score: {score:.4f}")
    scores.append(score)

    fold_no += 1

# Average score across folds
print(f"Average R2 Score: {sum(scores) / len(scores):.4f}")


In [ ]:
model.predict(X_test)